<div style="color:#007acc; text-align:center; font-size:32px; font-weight:bold; font-family:Arial, sans-serif;">
  Explore Data Analysis and Transform Equipenhance Log Data
</div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df=pd.read_csv("../ingest_from_ITS/data/m952_equipenhance_2025-05-04_00-00-00_to_2025-06-19_00-00-00.csv") 
df

C:\Users\GOSU.GOSU-TT-ZION1\AppData\Local\Temp\ipykernel_14440\57060766.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../ingest_from_ITS/data/m952_equipenhance_2025-05-04_00-00-00_to_2025-06-19_00-00-00.csv")


,@timestamp,_version,agent.name,labels.app,labels.publisher,ecs.version,related.user,event.ingested,event.original,event.kind,event.module,event.action,event.its.properties.i_before_level,event.its.properties.item_id,event.its.properties.level,event.its.properties.v_game_app_id,event.its.properties.i_profession,event.its.properties.character_id,event.its.properties.server_id,event.its.properties.item_uid,event.its.properties.v_role_id_tmp,event.its.properties.i_power_point,event.its.properties.trans_tag,event.its.properties.i_zone_area_id,event.its.properties.i_after_level,event.id,event.dataset,user.id,device.its.type,user_agent.os.name
0,2025-05-03T17:00:00.000Z,2,equipenhance,m952,gamo,8.9.0,['2407152127470964'],2025-06-03T10:14:23.073585271Z,"{""TransTag"":""TPID0202GSN108TM68164B90SQ4D34"",""GameSvrId"":""202"",""dtEventTime"":""2025-05-03T17:00:00.000Z"",""vGameAppid"":""1105309683"",""PlatID"":1,""iZoneAreaID"":202,""vopenid"":""2407152127470964"",""vRoleID"":23081817414240400,""vRoleIDTmp"":23081817414240400,""iLevel"":72,""iProfession"":2,""iPowerPoint"":13865544,""uItemId"":109001122,""uItemUid"":7495987460515185000,""iBeforeLevel"":2,""iAfterLevel"":3,""_id"":""MjMwODE4MTc0MTQyNDA0MDAmMjAyNS0wNS0wM1QxNzowMDowMC4wMDBa""}",event,sources,equipenhance,2,109001122,72,1105309683,2,23081817414240400,202,7495987460515185000,23081817414240400,13865544,TPID0202GSN108TM68164B90SQ4D34,202,3,MjMwODE4MTc0MTQyNDA0MDAmMjAyNS0wNS0wM1QxNzowMDowMC4wMDBa,track,2407152127470964,iOS,iOS
1,2025-05-03T17:00:00.000Z,2,equipenhance,m952,gamo,8.9.0,['2501151225298848'],2025-06-03T10:14:23.073795224Z,"{""TransTag"":""TPID0201GSN101TM68164B90SQ6B1B"",""GameSvrId"":""201"",""dtEventTime"":""2025-05-03T17:00:00.000Z"",""vGameAppid"":""1105309683"",""PlatID"":1,""iZoneAreaID"":201,""vopenid"":""2501151225298848"",""vRoleID"":644860036673072800,""vRoleIDTmp"":644860036673072800,""iLevel"":74,""iProfession"":2,""iPowerPoint"":15603305,""uItemId"":101031123,""uItemUid"":7499760836782148000,""iBeforeLevel"":0,""iAfterLevel"":1,""_id"":""NjQ0ODYwMDM2NjczMDcyODAwJjIwMjUtMDUtMDNUMTc6MDA6MDAuMDAwWg==""}",event,sources,equipenhance,0,101031123,74,1105309683,2,644860036673072800,201,7499760836782148000,644860036673072800,15603305,TPID0201GSN101TM68164B90SQ6B1B,201,1,NjQ0ODYwMDM2NjczMDcyODAwJjIwMjUtMDUtMDNUMTc6MDA6MDAuMDAwWg==,track,2501151225298848,iOS,iOS
2,2025-05-03T17:00:01.000Z,2,equipenhance,m952,gamo,8.9.0,['2503222323048092'],2025-06-03T10:14:23.073901462Z,"{""TransTag"":""TPID0202GSN204TM68164B91SQ347E"",""GameSvrId"":""202"",""dtEventTime"":""2025-05-03T17:00:01.000Z"",""vGameAppid"":""1105309683"",""PlatID"":2,""iZoneAreaID"":202,""vopenid"":""2503222323048092"",""vRoleID"":627971542367439200,""vRoleIDTmp"":627971542367439200,""iLevel"":49,""iProfession"":2,""iPowerPoint"":1733855,""uItemId"":106031018,""uItemUid"":7493726448533524000,""iBeforeLevel"":2,""iAfterLevel"":0,""_id"":""NjI3OTcxNTQyMzY3NDM5MjAwJjIwMjUtMDUtMDNUMTc6MDA6MDEuMDAwWg==""}",event,sources,equipenhance,2,106031018,49,1105309683,2,627971542367439200,202,7493726448533524000,627971542367439200,1733855,TPID0202GSN204TM68164B91SQ347E,202,0,NjI3OTcxNTQyMzY3NDM5MjAwJjIwMjUtMDUtMDNUMTc6MDA6MDEuMDAwWg==,track,2503222323048092,Android,Android
3,2025-05-03T17:00:01.000Z,1,equipenhance,m952,gamo,8.9.0,['2412231548057710'],2025-06-03T10:30:36.782537078Z,"{""TransTag"":""TPID0101GSN102TM68164B91SQ3F00"",""GameSvrId"":""101"",""dtEventTime"":""2025-05-03T17:00:01.000Z"",""vGameAppid"":""1105309683"",""PlatID"":2,""iZoneAreaID"":101,""vopenid"":""2412231548057710"",""vRoleID"":336925982649130560,""vRoleIDTmp"":336925982649130560,""iLevel"":68,""iProfession"":2,""iPowerPoint"":5009031,""uItemId"":101011222,""uItemUid"":7499541003169521000,""iBeforeLevel"":1,""iAfterLevel"":2,""_id"":""MzM2OTI1OTgyNjQ5MTMwNTYwJjIwMjUtMDUtMDNUMTc6MDA6MDEuMDAwWg==""}",event,sources,equipenhance,1,101011222,68,1105309683,2,336925982649130560,101,7499541003169521000,336925982649130560,5009031,TPID0101GSN102TM68164B9

In [3]:
df['user.id']=df['user.id'].astype(str)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820891 entries, 0 to 6820890
Data columns (total 30 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   @timestamp                           object
 1   _version                             int64 
 2   agent.name                           object
 3   labels.app                           object
 4   labels.publisher                     object
 5   ecs.version                          object
 6   related.user                         object
 7   event.ingested                       object
 8   event.original                       object
 9   event.kind                           object
 10  event.module                         object
 11  event.action                         object
 12  event.its.properties.i_before_level  int64 
 13  event.its.properties.item_id         int64 
 14  event.its.properties.level           int64 
 15  event.its.properties.v_game_app_id   int64 
 16  

In [5]:
df.describe()

,_version,event.its.properties.i_before_level,event.its.properties.item_id,event.its.properties.level,event.its.properties.v_game_app_id,event.its.properties.i_profession,event.its.properties.character_id,event.its.properties.server_id,event.its.properties.item_uid,event.its.properties.v_role_id_tmp,event.its.properties.i_power_point,event.its.properties.i_zone_area_id,event.its.properties.i_after_level
count,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00,6820891.00
mean,1.61,1.69,133394546.17,68.45,1105309683.00,1.58,496125046075250880.00,163.72,7497080629424434176.00,496125046075250880.00,15525271.80,163.72,2.00
std,14.33,1.72,117175004.69,16.00,0.00,0.56,335985194821229312.00,48.12,160677340970377216.00,335985194821229312.00,13749286.04,48.12,1.68
min,1.00,0.00,0.00,1.00,1105309683.00,1.00,281910488379979.00,101.00,0.00,281910488379979.00,22265.00,101.00,-1.00
25%,1.00,0.00,104011123.00,60.00,1105309683.00,1.00,196751873749834976.00,104.00,7498002714998463488.00,196751873749834976.00,4466754.00,104.00,1.00
50%,1.00,1.00,106031023.00,71.00,1105309683.00,2.00,456552869218959680.00,201.00,7503351373070282752.00,456552869218959680.00,11814780.00,201.00,2.00
75%,1.00,3.00,107001024.00,80.00,1105309683.00,2.00,773212626053584896.00,202.00,7508292005484201984.00,773212626053584896.00,22580858.50,202.00,3.00
max,781.00,14.00,1750223048.00,90.00,1105309683.00,4.00,1152640907553097600.00,204.00,7517334215164910592.00,1152640907553097600.00,62841816.00,204.00,16.00


In [6]:
df.isnull().sum()

@timestamp                                   0
_version                                     0
agent.name                                   0
labels.app                                   0
labels.publisher                             0
ecs.version                                  0
related.user                                 0
event.ingested                               0
event.original                               0
event.kind                                   0
event.module                                 0
event.action                                 0
event.its.properties.i_before_level          0
event.its.properties.item_id                 0
event.its.properties.level                   0
event.its.properties.v_game_app_id           0
event.its.properties.i_profession            0
event.its.properties.character_id            0
event.its.properties.server_id               0
event.its.properties.item_uid                0
event.its.properties.v_role_id_tmp           0
event.its.pro

In [7]:
null_ratio = df.isnull().mean()
cols_to_drop = null_ratio[null_ratio > 0.95].index
print("column deleted:", list(cols_to_drop))
df = df.drop(columns=cols_to_drop)
df.shape

column deleted: []


(6820891, 30)

In [8]:
df['@timestamp'] = pd.to_datetime(df['@timestamp'], errors='coerce', utc=True)
df.shape

(6820891, 30)

In [9]:
len(df['user.id'].unique())

55138

#### **Visualize**